# Goal

Get the first and last authorship for all CHW authors. Also apply my custom function for weighting authorship position. My custom weight function is defined by:
$$
    f(k,n,b,c) = 1 - c\left( \sigma(b(k-2)) - \sigma(b(k-(n-1))) \right),
$$
where 
- $k$ is the author position, 
- $n$ is the number of authors, 
- $b$ controls the steepness of the trough (I set $b = 4$), 
- $c$ is the minimum weight you want to assign to middle authors (in my case I set $c = (n-1)/n$ to give a minimum weight of 1/n), 
- and $\sigma$ is the logistic/sigmoid function.

In [3]:
import numpy as np
import pandas as pd
import string
import re
import pickle
import utils.author_metrics as am

import pybliometrics
from pybliometrics.scopus import AuthorRetrieval
import json
import requests

import plotly.express as px

In [4]:
df_auth = pd.read_excel('../Datasets/chw_ids.xlsx')
df_pub = pd.read_csv('../Datasets/chw_publications.csv')

# remove O'Brien to focus on CHW authors
df_auth = df_auth[df_auth['OBRIEN'] == 0]

df_auth.info()
df_pub.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2461 entries, 0 to 2727
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   Auid      2461 non-null   int64
 1   ACHRI     2461 non-null   int64
 2   BCCHR     2461 non-null   int64
 3   CHRIM     2461 non-null   int64
 4   WCHRI     2461 non-null   int64
 5   Sickkids  2461 non-null   int64
 6   CHUSJ     2461 non-null   int64
 7   OBRIEN    2461 non-null   int64
 8   MCMASTER  2461 non-null   int64
dtypes: int64(9)
memory usage: 192.3 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219997 entries, 0 to 219996
Data columns (total 35 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   CHW author           219997 non-null  int64  
 1   eid                  219997 non-null  object 
 2   doi                  205160 non-null  object 
 3   pii                  59958 non-null   object 
 4   pubmed_id            1793

# Getting the individual author ids

In [5]:
df_pub.loc[:, 'auids'] = df_pub.loc[:, 'author_ids'].str.split(';')

# remove any missing rows
df_pub = df_pub[~df_pub.author_ids.isna()]

# convert auids to int
df_pub['auids'] = df_pub['auids'].apply(lambda x: [int(i) for i in x])

# Get author positions

In [6]:
# get the CHW author position - come back to see the ones not found in their own author lists
not_found = []

for i in df_pub.index:
    try:
        df_pub.loc[i, 'chw_author_position'] = df_pub.loc[i, 'auids'].index(df_pub.loc[i, 'CHW author'])
    except ValueError:
        not_found.append(i)
        
print('{} authors not found in their own author lists\n'.format(len(not_found)))

# get first author positions
df_pub['first_author'] = df_pub['chw_author_position'].apply(lambda x: 1 if x == 0 else 0)

# get last author positions
df_pub['last_author'] = np.where(df_pub['chw_author_position'] == df_pub['author_count'] - 1, 1, 0)

# compute author weights with custom function
cant_compute = []

for i in df_pub.index:
    try:
        df_pub.loc[i, 'author_weight'] = am.trough(df_pub.loc[i, 'chw_author_position'], df_pub.loc[i, 'author_count'])
    except ValueError:
        cant_compute.append(i)
        
print('{} author weights could not be computed\n'.format(len(cant_compute)))

df_pub.head()

1344 authors not found in their own author lists

0 author weights could not be computed



,CHW author,eid,doi,pii,pubmed_id,title,subtype,subtypeDescription,creator,afid,...,citedby_count,openaccess,fund_acr,fund_no,fund_sponsor,auids,chw_author_position,first_author,last_author,author_weight
0,6503856044,2-s2.0-85102936784,10.1177/10935266211001986,NaN,33749384.0,Histologic Correlates of Molecular Group 4 Ped...,re,Review,Triscott J.,60193402;60023077;60020486;60018471;60010365;1...,...,1,0,MSFHR,undefined,Hannah's Heroes Foundation,"[53882034800, 7102134322, 7403550284, 71018199...",4.0,0,0,0.125299
1,6503856044,2-s2.0-85108228079,10.2217/pme-2020-0120,NaN,33973801.0,Deliberations about clinical pharmacogenetic t...,re,Review,Issa A.M.,60191838;60086967;60030835;60029502;60026553;6...,...,0,0,CIHR,1-155872,Genome Canada,"[7006615788, 57200159420, 6603414671, 36980983...",5.0,0,0,0.100006
2,6503856044,2-s2.0-85104934944,10.1002/cam4.3910,NaN,33939292.0,Trametinib therapy for children with neurofibr...,ar,Article,Ronsley R.,60023077;60010365,...,1,1,NaN,undefined,NaN,"[35272975600, 57204041913, 54388707200, 650385...",3.0,0,0,0.077233
3,6503856044,2-s2.0-85109755328,10.9778/cmajo.20200212,NaN,34021014.0,Family attitudes about and experiences with me...,ar,Article,Gibbard M.,60086967,...,2,1,NaN,undefined,NaN,"[57224506700, 57209177557, 6503856044, 6506014...",2.0,0,0,0.638490
4,6503856044,2-s2.0-85101694285,10.1016/S2352-4642(21)00020-1,S2352464221000201,33581749.0,Prevalence and risk factors for cisplatin-indu...,ar,Article,Moke D.J.,60086967;60030769;60029311;60023077;60020565;6...,...,5,0,NIH,P30CA014089,National Institutes of Health,"[57194505277, 57205079186, 24367129900, 162306...",4.0,0,0,0.076929


In [6]:
# Nov 22 - saving my work so far for now
df_pub.to_csv('pca_prep_wip.csv', index = False)

# Get publication counts by type

In [7]:
df = pd.concat(
    [
        df_pub,
        pd.get_dummies(df_pub['subtypeDescription'])
    ],
    axis = 1
)

# Get SCImago Journal Rank (SJR) for publication sources

In [8]:
journals = pd.read_csv('../Datasets/scimag_journals2020.csv', delimiter = ';', low_memory = False)
journals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32952 entries, 0 to 32951
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Rank                    32952 non-null  int64 
 1   Sourceid                32952 non-null  int64 
 2   Title                   32952 non-null  object
 3   Type                    32952 non-null  object
 4   Issn                    32952 non-null  object
 5   SJR                     32604 non-null  object
 6   SJR Best Quartile       32952 non-null  object
 7   H index                 32952 non-null  int64 
 8   Total Docs. (2020)      32952 non-null  int64 
 9   Total Docs. (3years)    32952 non-null  int64 
 10  Total Refs.             32952 non-null  int64 
 11  Total Cites (3years)    32952 non-null  int64 
 12  Citable Docs. (3years)  32952 non-null  int64 
 13  Cites / Doc. (2years)   32952 non-null  object
 14  Ref. / Doc.             32952 non-null  object
 15  Co

In [9]:
# get three columns of interest
df_jrn = journals[['Title', 'SJR', 'Categories', 'H index']].rename(columns = {'H index': 'journal_h_index'})

In [10]:
df2 = df.merge(df_jrn, left_on = 'publicationName', right_on = 'Title')

# convert 0,xx to 0.xx
df2['SJR'] = df2['SJR'].apply(am.convert_eu_to_float)

df2.head()

,CHW author,eid,doi,pii,pubmed_id,title,subtype,subtypeDescription,creator,afid,...,Erratum,Letter,Note,Retracted,Review,Short Survey,Title,SJR,Categories,journal_h_index
0,6503856044,2-s2.0-85102936784,10.1177/10935266211001986,NaN,33749384.0,Histologic Correlates of Molecular Group 4 Ped...,re,Review,Triscott J.,60193402;60023077;60020486;60018471;60010365;1...,...,0,0,0,0,1,0,Pediatric and Developmental Pathology,0.477,"Pediatrics, Perinatology and Child Health (Q2)...",60
1,6503856044,2-s2.0-85107850881,10.1177/10935266211021222,NaN,NaN,Malignant Mesothelioma With EWSR1-ATF1 Fusion ...,ar,Article,Ren H.,60028379;60023077;60021600;60019490;60008667,...,0,0,0,0,0,0,Pediatric and Developmental Pathology,0.477,"Pediatrics, Perinatology and Child Health (Q2)...",60
2,6507368744,2-s2.0-84873023233,10.2350/12-05-1204-CR.1,NaN,22901100.0,Myofibroblastoma: Report of a rare entity in t...,ar,Article,Chami R.,60030319;60016849,...,0,0,0,0,0,0,Pediatric and Developmental Pathology,0.477,"Pediatrics, Perinatology and Child Health (Q2)...",60
3,6601955128,2-s2.0-85027123847,10.1177/1093526616686008,NaN,28326958.0,Persistent left superior vena cava: A marker f...,ar,Article,Nair V.,60176051;60002306;100708789,...,0,0,0,0,0,0,Pediatric and Developmental Pathology,0.477,"Pediatrics, Perinatology and Child Health (Q2)...",60
4,6602178790,2-s2.0-84873023233,10.2350/12-05-1204-CR.1,NaN,22901100.0,Myofibroblastoma: Report of a rare entity in t...,ar,Article,Chami R.,60030319;60016849,...,0,0,0,0,0,0,Pediatric and Developmental Pathology,0.477,"Pediatrics, Perinatology and Child Health (Q2)...",60


In [11]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 193333 entries, 0 to 193332
Data columns (total 58 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   CHW author           193333 non-null  int64  
 1   eid                  193333 non-null  object 
 2   doi                  183710 non-null  object 
 3   pii                  54298 non-null   object 
 4   pubmed_id            161745 non-null  float64
 5   title                193331 non-null  object 
 6   subtype              193333 non-null  object 
 7   subtypeDescription   193252 non-null  object 
 8   creator              193319 non-null  object 
 9   afid                 189148 non-null  object 
 10  affilname            189148 non-null  object 
 11  affiliation_city     188861 non-null  object 
 12  affiliation_country  189114 non-null  object 
 13  author_count         193333 non-null  float64
 14  author_names         193333 non-null  object 
 15  author_ids       

# Group the publication statistics by author

In [12]:
# group the metrics by author
df3 = df2.groupby('CHW author', as_index = False).agg(
    {
        'author_count': ['median', 'mean'],
        'citedby_count': ['median', 'mean'],
        'citedby_count': 'sum',
        'openaccess': 'sum',
        'chw_author_position': ['median', 'mean'],
        'first_author': 'sum',
        'last_author': 'sum',
        'author_weight': ['median', 'mean'],
        'Article': 'sum',
        'Book': 'sum',
        'Book Chapter': 'sum',
        'Chapter': 'sum',
        'Conference Paper': 'sum',
        'Data Paper': 'sum',
        'Editorial': 'sum',
        'Erratum': 'sum',
        'Letter': 'sum',
        'Note': 'sum',
        'Retracted': 'sum',
        'Review': 'sum',
        'Short Survey': 'sum',
        'SJR': ['median', 'mean'],
        'journal_h_index': ['median', 'mean'],
    }
)

# change the multiindex columns to a simple index
df3.columns = [' '.join(col).strip() for col in df3.columns.values]

df3.info()
df3.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2720 entries, 0 to 2719
Data columns (total 28 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   CHW author                  2720 non-null   int64  
 1   author_count median         2720 non-null   float64
 2   author_count mean           2720 non-null   float64
 3   citedby_count sum           2720 non-null   int64  
 4   openaccess sum              2720 non-null   int64  
 5   chw_author_position median  2710 non-null   float64
 6   chw_author_position mean    2710 non-null   float64
 7   first_author sum            2720 non-null   int64  
 8   last_author sum             2720 non-null   int64  
 9   author_weight median        2710 non-null   float64
 10  author_weight mean          2710 non-null   float64
 11  Article sum                 2720 non-null   float64
 12  Book sum                    2720 non-null   uint8  
 13  Book Chapter sum            2720 

,CHW author,author_count median,author_count mean,citedby_count sum,openaccess sum,chw_author_position median,chw_author_position mean,first_author sum,last_author sum,author_weight median,...,Erratum sum,Letter sum,Note sum,Retracted sum,Review sum,Short Survey sum,SJR median,SJR mean,journal_h_index median,journal_h_index mean
0,6503856044,10.0,11.534091,2176,32,3.0,4.840909,4,6,0.181935,...,2,6,1,0,13,0,1.116,2.129307,105.0,145.443182
1,6503905253,8.0,9.120000,1214,13,2.0,3.800000,8,2,0.562500,...,1,0,0,0,4,0,1.779,2.286880,186.0,223.680000
2,6503911165,7.5,25.812500,518,2,4.0,15.812500,3,1,0.177038,...,0,0,0,0,0,0,1.082,1.509500,120.5,171.750000
3,6504148317,4.0,12.000000,359,0,2.5,5.833333,0,3,0.638490,...,0,0,0,0,1,0,0.805,3.676500,99.5,155.666667
4,6504169150,7.0,10.744186,1333,20,2.0,2.209302,16,4,0.583338,...,3,1,2,0,6,0,0.949,1.365558,60.0,100.883721


# Get author profiles

In [13]:
df_profiles = pd.read_excel('../Datasets/chw_author_profiles.xlsx')
df_profiles.info()
df_profiles.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2775 entries, 0 to 2774
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   affiliation_current    2775 non-null   object 
 1   affiliation_history    2773 non-null   object 
 2   alias                  0 non-null      float64
 3   citation_count         2775 non-null   int64  
 4   cited_by_count         2775 non-null   int64  
 5   coauthor_count         2775 non-null   int64  
 6   classificationgroup    2775 non-null   object 
 7   coauthor_link          2775 non-null   object 
 8   date_created           2775 non-null   object 
 9   document_count         2775 non-null   int64  
 10  eid                    2775 non-null   object 
 11  given_name             2775 non-null   object 
 12  h_index                2775 non-null   int64  
 13  historical_identifier  2423 non-null   object 
 14  identifier             2775 non-null   int64  
 15  inde

,affiliation_current,affiliation_history,alias,citation_count,cited_by_count,coauthor_count,classificationgroup,coauthor_link,date_created,document_count,...,orcid,publication_range,scopus_author_link,search_link,self_link,status,subject_areas,surname,url,Auid
0,"[Affiliation(id=103567372, parent=None, type='...","[Affiliation(id=107948211, parent=60010365, ty...",NaN,2219,1661,516,"[(1303, 1), (1313, 6), (2737, 1), (1311, 11), ...",http://api.elsevier.com/content/search/author?...,"(2005, 11, 28)",96,...,0000-0001-8005-2641,"(1998, 2021)",http://api.elsevier.com/content/author/author_...,http://api.elsevier.com/content/search/scopus?...,https://www.scopus.com/authid/detail.uri?partn...,update,"[Subjectarea(area='Biochemistry', abbreviation...",Rassekh,http://api.elsevier.com/content/author/author_...,6503856044
1,"[Affiliation(id=113080528, parent=60018491, ty...","[Affiliation(id=60008667, parent=None, type='p...",NaN,1314,1043,129,"[(1300, 5), (2712, 2), (1109, 1), (2808, 1), (...",http://api.elsevier.com/content/search/author?...,"(2005, 12, 1)",32,...,0000-0002-2079-8393,"(2004, 2021)",http://api.elsevier.com/content/author/author_...,http://api.elsevier.com/content/search/scopus?...,https://www.scopus.com/authid/detail.uri?partn...,update,"[Subjectarea(area='Biochemistry, Genetics and ...",Provençal,http://api.elsevier.com/content/author/author_...,6503905253
2,"[Affiliation(id=60009175, parent=None, type='p...","[Affiliation(id=60009175, parent=None, type='p...",NaN,590,582,293,"[(2740, 1), (2906, 1), (2746, 1), (2919, 1), (...",http://api.elsevier.com/content/search/author?...,"(2005, 12, 1)",21,...,NaN,"(2000, 2021)",http://api.elsevier.com/content/author/author_...,http://api.elsevier.com/content/search/scopus?...,https://www.scopus.com/authid/detail.uri?partn...,update,[Subjectarea(area='Pulmonary and Respiratory M...,Hendson,http://api.elsevier.com/content/author/author_...,6503911165
3,"[Affiliation(id=108988291, parent=60030319, ty...","[Affiliation(id=113069744, parent=60016849, ty...",NaN,302,272,49,"[(2746, 7), (2735, 7), (2726, 1), (1707, 1), (...",http://api.elsevier.com/content/search/author?...,"(2005, 12, 2)",14,...,NaN,"(2000, 2018)",http://api.elsevier.com/content/author/author_...,http://api.elsevier.com/content/search/scopus?...,https://www.scopus.com/authid/detail.uri?partn...,update,"[Subjectarea(area='Surgery', abbreviation='MED...",Himidan,http://api.elsevier.com/content/author/author_...,6504203692
4,"[Affiliation(id=112856410, parent=60002306, ty...","[Affiliation(id=109401138, parent=60002306, ty...",NaN,294,282,100,"[(2800, 2), (2723, 1), (2701, 1), (2746, 13), ...",http://api.elsevier.com/content/search/author?...,"(2005, 12, 2)",35,...,NaN,"(2004, 2021)",http://api.elsevier.com/content/author/author_...,http://api.elsevier.com/content/search/scopus?...,https://www.scopus.com/authid/detail.uri?partn...,update,"[Subjectarea(area='Neuroscience (all)', abbrev...",Ramien,http://api.elsevier.com/content/author/author_...,6504324290


In [14]:
# count the past and present affiliations
df_profiles['total_current_affiliations'] = df_profiles['affiliation_current'].str.findall('(Affiliation)\(').apply(am.count_affils)
df_profiles['total_past_affiliations'] = df_profiles['affiliation_history'].str.findall('(Affiliation)\(').apply(am.count_affils)

# get the number of years publishing
df_profiles['first_publication'] = df_profiles['publication_range'].str.extract('\(([0-9]*), ').astype(int)
df_profiles['last_publication'] = df_profiles['publication_range'].str.extract(', ([0-9]*)\)').astype(int)

df_profiles['publication_duration'] = df_profiles['last_publication'] - df_profiles['first_publication']

# get dummy variable for whether the orcid was reported or not
df_profiles['ORCID'] = np.where(df_profiles['orcid'].notna(), 1, 0)

df_profiles.head()

,affiliation_current,affiliation_history,alias,citation_count,cited_by_count,coauthor_count,classificationgroup,coauthor_link,date_created,document_count,...,subject_areas,surname,url,Auid,total_current_affiliations,total_past_affiliations,first_publication,last_publication,publication_duration,ORCID
0,"[Affiliation(id=103567372, parent=None, type='...","[Affiliation(id=107948211, parent=60010365, ty...",NaN,2219,1661,516,"[(1303, 1), (1313, 6), (2737, 1), (1311, 11), ...",http://api.elsevier.com/content/search/author?...,"(2005, 11, 28)",96,...,"[Subjectarea(area='Biochemistry', abbreviation...",Rassekh,http://api.elsevier.com/content/author/author_...,6503856044,1,32,1998,2021,23,1
1,"[Affiliation(id=113080528, parent=60018491, ty...","[Affiliation(id=60008667, parent=None, type='p...",NaN,1314,1043,129,"[(1300, 5), (2712, 2), (1109, 1), (2808, 1), (...",http://api.elsevier.com/content/search/author?...,"(2005, 12, 1)",32,...,"[Subjectarea(area='Biochemistry, Genetics and ...",Provençal,http://api.elsevier.com/content/author/author_...,6503905253,2,16,2004,2021,17,1
2,"[Affiliation(id=60009175, parent=None, type='p...","[Affiliation(id=60009175, parent=None, type='p...",NaN,590,582,293,"[(2740, 1), (2906, 1), (2746, 1), (2919, 1), (...",http://api.elsevier.com/content/search/author?...,"(2005, 12, 1)",21,...,[Subjectarea(area='Pulmonary and Respiratory M...,Hendson,http://api.elsevier.com/content/author/author_...,6503911165,1,14,2000,2021,21,0
3,"[Affiliation(id=108988291, parent=60030319, ty...","[Affiliation(id=113069744, parent=60016849, ty...",NaN,302,272,49,"[(2746, 7), (2735, 7), (2726, 1), (1707, 1), (...",http://api.elsevier.com/content/search/author?...,"(2005, 12, 2)",14,...,"[Subjectarea(area='Surgery', abbreviation='MED...",Himidan,http://api.elsevier.com/content/author/author_...,6504203692,2,8,2000,2018,18,0
4,"[Affiliation(id=112856410, parent=60002306, ty...","[Affiliation(id=109401138, parent=60002306, ty...",NaN,294,282,100,"[(2800, 2), (2723, 1), (2701, 1), (2746, 13), ...",http://api.elsevier.com/content/search/author?...,"(2005, 12, 2)",35,...,"[Subjectarea(area='Neuroscience (all)', abbrev...",Ramien,http://api.elsevier.com/content/author/author_...,6504324290,2,24,2004,2021,17,0


In [15]:
df_profiles.columns

Index(['affiliation_current', 'affiliation_history', 'alias', 'citation_count',
       'cited_by_count', 'coauthor_count', 'classificationgroup',
       'coauthor_link', 'date_created', 'document_count', 'eid', 'given_name',
       'h_index', 'historical_identifier', 'identifier', 'indexed_name',
       'initials', 'name_variants', 'orcid', 'publication_range',
       'scopus_author_link', 'search_link', 'self_link', 'status',
       'subject_areas', 'surname', 'url', 'Auid', 'total_current_affiliations',
       'total_past_affiliations', 'first_publication', 'last_publication',
       'publication_duration', 'ORCID'],
      dtype='object')

In [16]:
df_profiles.drop(
    [
        'identifier', 
        'historical_identifier',
        'affiliation_current',
        'affiliation_history',
        'alias',
        'coauthor_link',
        'date_created',
        'eid',
        'given_name',
        'initials',
        'name_variants',
        'orcid',
        'scopus_author_link',
        'search_link',
        'self_link',
        'status',
        'surname',
        'url',
        'publication_range'
    ], 
    axis = 1, 
    inplace = True
)

df_profiles.head()

,citation_count,cited_by_count,coauthor_count,classificationgroup,document_count,h_index,indexed_name,subject_areas,Auid,total_current_affiliations,total_past_affiliations,first_publication,last_publication,publication_duration,ORCID
0,2219,1661,516,"[(1303, 1), (1313, 6), (2737, 1), (1311, 11), ...",96,22,Rassekh S.,"[Subjectarea(area='Biochemistry', abbreviation...",6503856044,1,32,1998,2021,23,1
1,1314,1043,129,"[(1300, 5), (2712, 2), (1109, 1), (2808, 1), (...",32,16,Provençal N.,"[Subjectarea(area='Biochemistry, Genetics and ...",6503905253,2,16,2004,2021,17,1
2,590,582,293,"[(2740, 1), (2906, 1), (2746, 1), (2919, 1), (...",21,12,Hendson L.,[Subjectarea(area='Pulmonary and Respiratory M...,6503911165,1,14,2000,2021,21,0
3,302,272,49,"[(2746, 7), (2735, 7), (2726, 1), (1707, 1), (...",14,7,Himidan S.,"[Subjectarea(area='Surgery', abbreviation='MED...",6504203692,2,8,2000,2018,18,0
4,294,282,100,"[(2800, 2), (2723, 1), (2701, 1), (2746, 13), ...",35,10,Ramien M.,"[Subjectarea(area='Neuroscience (all)', abbrev...",6504324290,2,24,2004,2021,17,0


# Merge the author profile stats with stats from publications

In [17]:
df_stats = pd.merge(df_profiles, df3, left_on = 'Auid', right_on = 'CHW author')
df_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2767 entries, 0 to 2766
Data columns (total 43 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   citation_count              2767 non-null   int64  
 1   cited_by_count              2767 non-null   int64  
 2   coauthor_count              2767 non-null   int64  
 3   classificationgroup         2767 non-null   object 
 4   document_count              2767 non-null   int64  
 5   h_index                     2767 non-null   int64  
 6   indexed_name                2767 non-null   object 
 7   subject_areas               2767 non-null   object 
 8   Auid                        2767 non-null   int64  
 9   total_current_affiliations  2767 non-null   int64  
 10  total_past_affiliations     2767 non-null   int64  
 11  first_publication           2767 non-null   int64  
 12  last_publication            2767 non-null   int64  
 13  publication_duration        2767 

# Merge with CHW ids to get affiliation

In [18]:
affils = pd.read_excel('../Datasets/chw_ids.xlsx')
affils.drop('OBRIEN', axis = 1, inplace = True)
affils.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2728 entries, 0 to 2727
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   Auid      2728 non-null   int64
 1   ACHRI     2728 non-null   int64
 2   BCCHR     2728 non-null   int64
 3   CHRIM     2728 non-null   int64
 4   WCHRI     2728 non-null   int64
 5   Sickkids  2728 non-null   int64
 6   CHUSJ     2728 non-null   int64
 7   MCMASTER  2728 non-null   int64
dtypes: int64(8)
memory usage: 170.6 KB


In [19]:
af = affils.melt(
    'Auid', 
    var_name = 'affil'
).query(
    'value == 1'
).groupby(
    'Auid'
)[
    'affil'
].apply(
    ', '.join
).reset_index()

af

,Auid,affil
0,6503856044,BCCHR
1,6503905253,BCCHR
2,6503911165,ACHRI
3,6504148317,MCMASTER
4,6504203692,Sickkids
...,...,...
2458,57248713000,BCCHR
2459,57249145700,MCMASTER
2460,57262851900,CHUSJ
2461,57266821900,Sickkids


In [20]:
df_stats = pd.merge(df_stats, af, on = 'Auid')
df_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2500 entries, 0 to 2499
Data columns (total 44 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   citation_count              2500 non-null   int64  
 1   cited_by_count              2500 non-null   int64  
 2   coauthor_count              2500 non-null   int64  
 3   classificationgroup         2500 non-null   object 
 4   document_count              2500 non-null   int64  
 5   h_index                     2500 non-null   int64  
 6   indexed_name                2500 non-null   object 
 7   subject_areas               2500 non-null   object 
 8   Auid                        2500 non-null   int64  
 9   total_current_affiliations  2500 non-null   int64  
 10  total_past_affiliations     2500 non-null   int64  
 11  first_publication           2500 non-null   int64  
 12  last_publication            2500 non-null   int64  
 13  publication_duration        2500 

# Save the data for PCA

In [21]:
df_stats.to_excel('../Datasets/author_stats_for_PCA_dec14.xlsx', index = False)